# Fake Data

This notebooks exists to perform a creation of a fake dataset using probability to generate our rows.

The rules to consider:
-  Our model is a star schema
-  We will provide a **hard** manipulation in dimensions table
-  In fact tables we will calculate a intentional __p__ for every possibility in a list

---

To do this we will use a numpy function named **random** and the **choice** method.

In [1]:
import pandas as pd
import numpy as np

#### Function to get available items based in items and dimensions 

In [2]:
def items_available(items, item_label, item_dim, max_repeat):
    """
    items: items recurring list
    item_label: item label the function will build the groupby count
    item_dim: dimension dataframe with all items properties
    max_repeat: how many times each item dim needs to appear in target df
    """
    group_count_item = items.groupby(item_label).count()[[items.columns[1]]]
    group_count_item = group_count_item.rename(columns={group_count_item.columns[0]: 'count'})
    group_count_item = item_dim.merge(group_count_item, how='left', on=item_label)

    t = group_count_item[group_count_item['count'] != max_repeat][item_label].to_list()
    return t

## Names dataset

We will use contoso clients (free data from microsoft) to use their fake informations (like names etc.)

In [9]:
# df_names = pd.read_excel('../pbi-4-business/fakedata/names.xlsx')
df_names = pd.read_excel('c:/Users/Cicatr1z66/dev/pbi-4-business/fakedata/names.xlsx')

## Data Model

## Create dimensions

#### Supervisors Dimension

-  10 supervisors/lines
-  Primary key is an np.arrange(10)

In [10]:
# df_supervisors['supervisor_id'] -> identity 1 to 10
supervisor_id = pd.Series(np.arange(1, 11))
supervisor_name = pd.Series(df_names.iloc[:10]['name'])

dim_supervisors = pd.DataFrame({'supervisor_id': supervisor_id.values, 
                                'supervisor_name': supervisor_name.values})
dim_supervisors.head(5)

,supervisor_id,supervisor_name
0,1,Kim Abercrombie
1,2,Sagiv Hadaya
2,3,Luka Abrus
3,4,Kirk Nason
4,5,Humberto Acevedo


#### Analysts Dimension

-  50 analysts
-  5 analysts per supervisor

In [12]:
# create analyst_id and analyst name
analyst_id = pd.Series(np.arange(1, 51))
analyst_name = pd.Series(df_names.iloc[11:61]['name'])

dim_analysts = pd.DataFrame({'analyst_id': analyst_id.values,
                             'analyst_name': analyst_name.values})

dim_analysts['supervisor_id'] = dim_analysts['analyst_id'].map(lambda x: 0)

# iterate over analysts df and fill random supervisors
analyst_range = range(len(dim_analysts.index))
for analyst in analyst_range:
    items_available_list = items_available(dim_analysts, 'supervisor_id', dim_supervisors, 5)
    random_sup = np.random.choice(items_available_list)
    dim_analysts.at[analyst, 'supervisor_id'] = random_sup

dim_analysts.head(5)

,analyst_id,analyst_name,supervisor_id
0,1,Michael Raheem,5
1,2,David Ahs,10
2,3,Miguel Saenz,2
3,4,Kim Akers,10
4,5,Kate Taneyhill,5


In [12]:
dim_analysts.to_csv('dim_analyts.csv', index=False)
dim_supervisors.to_csv('dim_supervisors.csv', index=False)

#### Locations

To create random locations we will use a municipios table

In [14]:
municipios = pd.read_excel('c:/Users/Cicatr1z66/dev/municipios.xlsx')
locations_sample = pd.DataFrame({'location_id': np.random.randint(len(municipios.index), size=6)})
dim_locations = locations_sample.merge(municipios,
                                       how='left',
                                       left_on='location_id',
                                       right_index=True)[['location_id', 'nome', 'latitude', 'longitude']]

In [15]:
dim_locations.head(5)

,location_id,nome,latitude,longitude
0,105,Alegria,-278345.0,-540557.0
1,3184,Morro Cabeça no Tempo,-971891.0,-439072.0
2,3504,Padre Bernardo,-151605.0,-482833.0
3,4858,São Pedro,-225483.0,-479096.0
4,2392,Itaporanga,-730202.0,-381504.0


#### Clientes Dimension

- 12 clients
- 2 per location

In [16]:
# create analyst_id and analyst name
client_id = pd.Series(np.arange(1, 13))
client_name = pd.Series(df_names.iloc[62:74]['name'])

dim_clients = pd.DataFrame({'client_id': client_id.values,
                            'client_name': client_name.values})

dim_clients['location_id'] = dim_clients['client_id'].map(lambda x: 0)

# iterate over dim clients and fill location ramdomly
client_range = range(len(dim_clients.index))

for client in client_range:
    items_available_list = items_available(dim_clients, 'location_id', dim_locations, 2)
    random_location = np.random.choice(items_available_list)
    dim_clients.at[client, 'location_id'] = random_location

dim_clients.head(5)

,client_id,client_name,location_id
0,1,Karolina Salas-Szlejter,105
1,2,Anna Bedecs,3504
2,3,Oliver Szimmetat,3184
3,4,Ann Beebe,105
4,5,Jeff Teper,3504


#### Dim Category

-  4 Category
  -  Perifericos
  -  Hardware
  -  Software
  -  Infra

In [ ]:
category_id = pd.Series(np.arange(1, 5))
category_name = pd.Series(['Perifericos', 'Hardware', 'Software', 'Infra'])

dim_category = pd.DataFrame({'category_id': category_id,
                             'category_name': category_name})


#### Dim Sub-Category

-  8 sub-category
-  2 per category

In [ ]:
sub_category_id = pd.Series(np.arange(1, 9))
sub_category_name = pd.Series(['Keyboard', 'Headset',
                               'Motherboard', 'RAM Memory',
                               'Visual Studio', 'Power BI',
                               'Chair', 'Desk'])
category_id = pd.Series([1, 1, 2, 2, 3, 3, 4, 4])

dim_sub_category = pd.DataFrame({'sub_category_id': sub_category_id,
                                 'sub_category_name': sub_category_name,
                                 'category_id': category_id})

In [ ]:
dim_analysts.to_csv('dim_analyts.csv', index=False)
dim_supervisors.to_csv('dim_supervisors.csv', index=False)
dim_clients.to_csv('dim_clients.csv', index=False)
dim_locations.to_csv('dim_locations.csv', index=False)
dim_category.to_csv('dim_category.csv', index=False)
dim_sub_category.to_csv('dim_sub_category.csv', index=False)
